#### 1.

In [108]:
import pandas as pd
import numpy as np
import sqlite3
from contextlib import closing

# HOMEWORK 1.

# Creating a function that connects to the database and accesses tables dynamically
def load_db_tables(db_path):
    """Load all tables from a SQLite database into a dictionary of DataFrames."""
    tables_dict = {}
    
    try:
        with closing(sqlite3.connect(db_path)) as conn:
            # Step 1: List all tables
            tables = pd.read_sql(
                "SELECT name FROM sqlite_master WHERE type='table';", 
                conn
            )['name'].tolist()
            
            # Step 2: Dynamically load each table into a DataFrame
            for table in tables:
                query = f"SELECT * FROM {table};"  # Dynamic query
                tables_dict[table] = pd.read_sql(query, conn)
            
            return tables_dict
    
    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
        raise
    except Exception as e:
        print(f"Unexpected error: {e}")
        raise

# database file path
db_path = "chinook.db"

# 'tables_dict' variable that 
tables_dict = load_db_tables(db_path)

# Task 1. Find the total amount spent by each customer on purchases (considering invoices).

invoices_df = tables_dict['invoices']  # From your dynamically loaded tables

# Calculate total spent per customer using Pandas
customer_spending = (
    invoices_df
    .groupby('CustomerId', as_index=False)  # Group by customer
    .agg(TotalSpent=('Total', 'sum'))       # Sum totals
    .sort_values('TotalSpent', ascending=False)  # Sort high-to-low
)

# The result dataframe
customer_spending

# Task 2. Identify the top 5 customers with the highest total purchase amounts.

# Top 5 customers with highest spending. We use 'customer_spending' df as it is already created
# we use 'nlargest()' function to specifically filter the top 5 though we could use 'customer_spending.head(5)' as it is alreadg sorted.
top5 = customer_spending.nlargest(5, 'TotalSpent') 

# The result data frame
top5


# Task 3. Display the customer ID, name, and the total amount spent for the top 5 customers.

# Get top 5 customers by total spending
top_customers = (
    tables_dict['invoices']
    .groupby('CustomerId', as_index=False)  # Group by customer
    .agg(TotalSpent=('Total', 'sum'))       # Sum purchase amounts
    .nlargest(5, 'TotalSpent')              # Select top 5
    .merge(
        tables_dict['customers'][['CustomerId', 'FirstName', 'LastName']],
        on='CustomerId',
        how='left'
    )
    .assign(CustomerName=lambda x: x['FirstName'] + ' ' + x['LastName'])
    .drop(columns=['FirstName', 'LastName'])
)

top_customers

# HOWMEWORK 2

# Album vs. Individual Track Purchases:



# Accessing 'tracks' table and saving in a variable
tracks_df = tables_dict['tracks']

# Accessing 'invoice_items' table from 'tables_dict'
invoices_cont = tables_dict['invoice_items']


# Step 1: Map tracks to albums
track_album = tracks_df[['TrackId', 'AlbumId']]

# Step 2: Merge invoices with track-album data
invoice_tracks = (
    invoices_cont.merge(invoices_df[['InvoiceId', 'CustomerId']], on='InvoiceId')
    .merge(track_album, on='TrackId')
)

# Step 3: Count unique tracks purchased per customer per album
customer_album_stats = (
    invoice_tracks
    .groupby(['CustomerId', 'AlbumId'])
    .agg(PurchasedTracks=('TrackId', 'nunique'))  # Unique tracks per customer/album
    .reset_index()
)

# Step 4: Add total tracks per album
customer_album_stats = customer_album_stats.merge(
    tracks_df.groupby('AlbumId')['TrackId'].nunique().reset_index(name='TotalTracks'),
    on='AlbumId'
)

# Step 5: Identify customers who bought ALL tracks for at least one album
# (These are "full album" buyers, even if they also bought individual tracks)
full_album_customers = (
    customer_album_stats
    .query('PurchasedTracks == TotalTracks')
    ['CustomerId']
    .unique()
)

# Step 6: Classify all customers
all_customers = invoices_df['CustomerId'].unique()
individual_track_only_customers = [c for c in all_customers if c not in full_album_customers]

# Step 7: Calculate percentages
total_customers = len(all_customers)
pct_individual = (len(individual_track_only_customers) / total_customers) * 100
pct_full_album = 100 - pct_individual

# Create summary
summary = pd.DataFrame({
    'Preference': ['Prefers Individual Tracks', 'Prefers Full Albums'],
    'Percentage': [pct_individual, pct_full_album],
    'CustomerCount': [len(individual_track_only_customers), len(full_album_customers)]
})

# The results
print("Customer Preference Summary:")
print(summary)

full_album_customers

Customer Preference Summary:
                  Preference  Percentage  CustomerCount
0  Prefers Individual Tracks    64.40678             38
1        Prefers Full Albums    35.59322             21


array([ 1,  2,  3,  4,  7, 13, 21, 24, 27, 33, 34, 35, 38, 39, 40, 41, 43,
       47, 51, 57, 58])